<a href="https://colab.research.google.com/github/qmaruf/fastai/blob/master/fastai_dl2_pascal_mrf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing necessary modules

In [4]:
import sys
print (sys.version)

import sys
print (sys.version)
! pip3 install fastai==0.7.0
! pip3 install torchtext==0.2.3

3.6.7 (default, Oct 22 2018, 11:32:17) 
[GCC 8.2.0]
3.6.7 (default, Oct 22 2018, 11:32:17) 
[GCC 8.2.0]
    100% |████████████████████████████████| 122kB 6.0MB/s 
    100% |████████████████████████████████| 3.6MB 7.4MB/s 
    100% |████████████████████████████████| 1.5MB 14.7MB/s 
    100% |████████████████████████████████| 496.4MB 28kB/s 
    100% |████████████████████████████████| 61kB 22.8MB/s 
    100% |████████████████████████████████| 12.4MB 2.1MB/s 
    100% |████████████████████████████████| 81kB 17.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/9f/78/26/fb8c0acb91a100dc8914bf236c4eaa4b207cb876893c40b745
  Stored in directory: /root/.cache/pip/wheels/85/7d/12/2dfa5c0195f921ac935f5e8f27deada74972edc0ae9988a9c1
Successfully built bcolz feather-format
mizani 0.5.3 has requirement pandas>=0.23.4, but you'll have pandas 0.22.0 which is incompatible.
plotnine 0.5.1 has requirement pandas>=0.23.4, but you'll have pandas 0.22.0 which is incompatible.
  Found existing installatio

    100% |████████████████████████████████| 51kB 3.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/42/a6/f4/b267328bde6bb680094a0c173e8e5627ccc99543abded97204
Successfully built torchtext
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [0]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [0]:
from fastai.conv_learner import *
from fastai.dataset import *

from pathlib import Path
import json
from PIL import ImageDraw, ImageFont
from matplotlib import patches, patheffects
torch.cuda.set_device(0)

In [4]:
'''
# Download data
!mkdir data
!wget http://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar -P data/
!wget https://storage.googleapis.com/coco-dataset/external/PASCAL_VOC.zip -P data/
!tar -xf data/VOCtrainval_06-Nov-2007.tar -C data/
!unzip data/PASCAL_VOC.zip -d data/
!rm -rf data/PASCAL_VOC.zip data/VOCtrainval_06-Nov-2007.tar
'''

'\n# Download data\n!mkdir data\n!wget http://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar -P data/\n!wget https://storage.googleapis.com/coco-dataset/external/PASCAL_VOC.zip -P data/\n!tar -xf data/VOCtrainval_06-Nov-2007.tar -C data/\n!unzip data/PASCAL_VOC.zip -d data/\n!rm -rf data/PASCAL_VOC.zip data/VOCtrainval_06-Nov-2007.tar\n'

In [7]:
PATH = Path('data/PASCAL_VOC/')
list(PATH.iterdir())

[PosixPath('data/PASCAL_VOC/pascal_test2007.json'),
 PosixPath('data/PASCAL_VOC/pascal_val2012.json'),
 PosixPath('data/PASCAL_VOC/pascal_train2007.json'),
 PosixPath('data/PASCAL_VOC/pascal_train2012.json'),
 PosixPath('data/PASCAL_VOC/pascal_val2007.json')]

In [37]:
train_json = json.load((PATH/'pascal_train2007.json').open())
print (train_json.keys())

dict_keys(['images', 'type', 'annotations', 'categories'])


In [0]:
IMAGES, TYPE, ANNOTATIONS, CATEGORIES = ['images', 'type', 'annotations', 'categories']

In [39]:
train_json[IMAGES][:5]

[{'file_name': '000012.jpg', 'height': 333, 'id': 12, 'width': 500},
 {'file_name': '000017.jpg', 'height': 364, 'id': 17, 'width': 480},
 {'file_name': '000023.jpg', 'height': 500, 'id': 23, 'width': 334},
 {'file_name': '000026.jpg', 'height': 333, 'id': 26, 'width': 500},
 {'file_name': '000032.jpg', 'height': 281, 'id': 32, 'width': 500}]

In [40]:
train_json[ANNOTATIONS][:2]

[{'area': 34104,
  'bbox': [155, 96, 196, 174],
  'category_id': 7,
  'id': 1,
  'ignore': 0,
  'image_id': 12,
  'iscrowd': 0,
  'segmentation': [[155, 96, 155, 270, 351, 270, 351, 96]]},
 {'area': 13110,
  'bbox': [184, 61, 95, 138],
  'category_id': 15,
  'id': 2,
  'ignore': 0,
  'image_id': 17,
  'iscrowd': 0,
  'segmentation': [[184, 61, 184, 199, 279, 199, 279, 61]]}]

In [41]:
train_json[CATEGORIES][:10]

[{'id': 1, 'name': 'aeroplane', 'supercategory': 'none'},
 {'id': 2, 'name': 'bicycle', 'supercategory': 'none'},
 {'id': 3, 'name': 'bird', 'supercategory': 'none'},
 {'id': 4, 'name': 'boat', 'supercategory': 'none'},
 {'id': 5, 'name': 'bottle', 'supercategory': 'none'},
 {'id': 6, 'name': 'bus', 'supercategory': 'none'},
 {'id': 7, 'name': 'car', 'supercategory': 'none'},
 {'id': 8, 'name': 'cat', 'supercategory': 'none'},
 {'id': 9, 'name': 'chair', 'supercategory': 'none'},
 {'id': 10, 'name': 'cow', 'supercategory': 'none'}]

In [0]:
id_to_name = {item['id']:item['name'] for item in train_json[CATEGORIES]}
id_to_filename = {item['id']:item['file_name'] for item in train_json[IMAGES]}
ids = [item['id'] for item in train_json[IMAGES]]

In [0]:
# images
#list((PATH/'VOCdevkit').iterdir())

In [0]:
train_jpegs = Path('./data/VOCdevkit/VOC2007/JPEGImages/')

In [0]:
#list(train_jpegs.iterdir())
#train_json[IMAGES]

In [0]:
def hw_bb(bb):
  xmin = bb[0]
  ymin = bb[1]
  xmax = 